In [ ]:
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

label_methods = [
    "random",
    "trend",
    "trend_profit",
    "trend_multi",
    "clusters",
    "multi_window",
    "validated_levels",
    "zigzag",
    "mean_rev",
    "mean_rev_multi",
    "mean_rev_vol",
    "filter",
    "multi_filter",
    "filter_bidirectional",
    "filter_one",
    "trend_one",
    "filter_flat",
]
search_methods = [
    'clusters',
    'markov',
    'lgmm',
    'wkmeans',
    'mapie',
    'causal',
]

configs = [
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='sucessive',
        train_start=datetime(2020,1,1), train_end=datetime(2025,1,1),
        test_start=datetime(2022,1,1),  test_end=datetime(2023,6,1),
        label_method='random', search_type='lgmm', search_subtype='',
        n_models=1, debug=False,
    ),
]

# Actualizar n_jobs en cada configuración
for cfg in configs:
    cfg["tag"] = f"{cfg['symbol']}_{cfg['timeframe']}_{cfg['direction']}_{cfg['label_method']}_{cfg['search_type']}_{cfg.get('search_subtype', '')}".strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None
if not study:
    n_trials = 5000
else:
    n_trials = len(study.trials) + 5000

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 27/5026 score=11.240593 avg=2.26s mem=610.30MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 28/5026 score=11.240593 avg=2.98s mem=604.75MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 29/5026 score=11.240593 avg=3.01s mem=618.52MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 30/5026 score=11.792066 avg=3.63s mem=638.56MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 31/5026 score=11.792066 avg=3.59s mem=687.67MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 32/5026 score=11.792066 avg=3.71s mem=689.18MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 33/5026 score=11.792066 avg=3.61s mem=689.18MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 34/5026 score=11.792066 avg=3.64s mem=702.14MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 35/5026 score=11.792066 avg=3.58s mem=702.17MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 36/5026 score=11.792066 avg=3.93s mem=667.15MB 
[XAUUSD_H1_buy_random_lgmm] modelo 0 trial 37/5026 score=11.792066 avg=3.91s mem=645.48MB 